<a href="https://colab.research.google.com/github/lawrencejesse/Sentinel2_Extractor/blob/main/Multi_Year_ProductivityRaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Create a Python notebook that allows the user to upload a polygon (KML/GeoJSON/SHP) and multiple NDVI rasters (GeoTIFF). The notebook should then:
1. Load the uploaded data.
2. Reproject and align all NDVI rasters to the grid of the first raster.
3. Clip the rasters to the polygon boundary.
4. Compute the mean NDVI within the polygon for each raster.
5. Calculate a per-year Productivity Index for each raster using the formula (NDVI / mean) * 100.
6. Calculate a Multi-Year Composite Productivity Index by averaging the per-year indices.
7. Create a 3-class map based on the Composite Productivity Index (below average, average, above average).
8. Save the per-year Productivity GeoTIFFs, the Composite GeoTIFF, the 3-class map GeoTIFF, and a CSV summary of the mean NDVI and productivity indices to a new folder in the notebook's memory.
The notebook should be broken down into discrete steps for easy troubleshooting.

## Setup

### Subtask:
Install necessary libraries (geopandas, rasterio, etc.) and create a directory for outputs.


**Reasoning**:
Install the necessary libraries using pip.



In [1]:
%pip install geopandas rasterio fiona shapely numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 25.9 MB/s eta 0:00:00


**Reasoning**:
Import the necessary libraries and create the output directory.



In [2]:
# Imports for both Jupyter and Colab environments
%pip install -q geopandas rasterio fiona shapely numpy

import geopandas
import rasterio
import rasterio.warp
import rasterio.mask
import fiona
from shapely.geometry import shape
import numpy as np
import os
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Detect environment (Colab vs Jupyter)
try:
    from google.colab import files
    IN_COLAB = True
    print("Running in Google Colab environment")
except ImportError:
    IN_COLAB = False
    print("Running in Jupyter environment")


Running in Google Colab environment


## Upload data

### Subtask:
Create widgets for uploading the AOI polygon and multiple NDVI rasters.


**Reasoning**:
Create and display the file upload widgets for the AOI polygon and NDVI rasters.



In [3]:
# Create configuration widgets
skip_alignment_widget = widgets.Checkbox(
    value=False,
    description='Skip Alignment',
    tooltip='Skip the reprojection and alignment step if rasters are already aligned'
)

display(skip_alignment_widget)
print("Configuration widgets created.")


Checkbox(value=False, description='Skip Alignment')

Configuration widgets created.


In [4]:
# Create file upload widgets
aoi_upload = widgets.FileUpload(
    accept='.kml,.geojson,.shp',
    multiple=False,
    description='Upload AOI Polygon (KML, GeoJSON, or SHP)'
)

ndvi_uploads = widgets.FileUpload(
    accept='.tif,.tiff',
    multiple=True,
    description='Upload NDVI Rasters (GeoTIFFs)'
)

display(aoi_upload)
display(ndvi_uploads)
print("File upload widgets created. Please upload your AOI polygon and NDVI rasters.")


FileUpload(value={}, accept='.kml,.geojson,.shp', description='Upload AOI Polygon (KML, GeoJSON, or SHP)')

FileUpload(value={}, accept='.tif,.tiff', description='Upload NDVI Rasters (GeoTIFFs)', multiple=True)

File upload widgets created. Please upload your AOI polygon and NDVI rasters.


## Load data

### Subtask:
Load the uploaded polygon and raster files into appropriate data structures (geopandas DataFrame and rasterio datasets).


**Reasoning**:
Load the uploaded AOI polygon and NDVI raster files into appropriate data structures using fiona and rasterio, handling cases where no files are uploaded.



In [5]:
aoi_gdf = None
original_ndvi_datasets = []
memfiles = []  # Keep MemoryFile objects alive

# Function to handle different widget structures (Jupyter vs Colab)
def get_file_content(upload_widget, is_multiple=False):
    """Extract file content from upload widget, handling both Jupyter and Colab formats"""
    files_data = []

    if upload_widget.value:
        # Check if it's Colab format (dict) or Jupyter format (tuple)
        if isinstance(upload_widget.value, dict):
            # Colab format: {'filename': {'content': bytes}}
            for filename, file_info in upload_widget.value.items():
                files_data.append({
                    'name': filename,
                    'content': file_info['content']
                })
        else:
            # Jupyter format: ({'name': str, 'content': bytes}, ...)
            for file_dict in upload_widget.value:
                files_data.append({
                    'name': file_dict['name'],
                    'content': file_dict['content']
                })

    return files_data[0] if files_data and not is_multiple else files_data

# Load AOI polygon
aoi_file = get_file_content(aoi_upload, is_multiple=False)
if aoi_file:
    try:
        with fiona.io.MemoryFile(aoi_file['content']) as memfile:
            aoi_gdf = geopandas.read_file(memfile)
        print(f"AOI polygon '{aoi_file['name']}' loaded successfully.")
        print(f"AOI CRS: {aoi_gdf.crs}")
    except Exception as e:
        print(f"Error loading AOI file: {e}")
else:
    print("No AOI file uploaded. Skipping AOI loading.")

# Load NDVI rasters
ndvi_files = get_file_content(ndvi_uploads, is_multiple=True)
if ndvi_files:
    for file_data in ndvi_files:
        try:
            # Keep MemoryFile alive by storing it
            memfile = rasterio.io.MemoryFile(file_data['content'])
            dataset = memfile.open()
            original_ndvi_datasets.append(dataset)
            memfiles.append(memfile)  # Important: Keep memfile alive!
            print(f"NDVI raster '{file_data['name']}' loaded successfully.")
        except Exception as e:
            print(f"Error loading NDVI file '{file_data['name']}': {e}")
else:
    print("No NDVI files uploaded. Skipping NDVI loading.")

# Initialize ndvi_datasets for later use
ndvi_datasets = []

# Summary
if aoi_gdf is not None and original_ndvi_datasets:
    print(f"\nSummary: Loaded {len(original_ndvi_datasets)} NDVI raster(s) and 1 AOI polygon.")
elif original_ndvi_datasets:
    print(f"\nSummary: Loaded {len(original_ndvi_datasets)} NDVI raster(s), no AOI polygon.")
elif aoi_gdf is not None:
    print(f"\nSummary: Loaded 1 AOI polygon, no NDVI rasters.")
else:
    print(f"\nSummary: No files loaded.")


AOI polygon 'NW-21-2-14w1FieldBoundary.kml' loaded successfully.
AOI CRS: EPSG:4326
NDVI raster 'NW-21-002-14W1M_2019-07-25_NDVI.tif' loaded successfully.
NDVI raster 'NW-21-002-14W1M_2019-07-30_NDVI.tif' loaded successfully.
NDVI raster 'NW-21-002-14W1M_2019-08-19_NDVI.tif' loaded successfully.
NDVI raster 'NW-21-002-14W1M_2020-07-14_NDVI.tif' loaded successfully.
NDVI raster 'NW-21-002-14W1M_2021-07-22_NDVI.tif' loaded successfully.
NDVI raster 'NW-21-002-14W1M_2021-07-27_NDVI.tif' loaded successfully.
NDVI raster 'NW-21-002-14W1M_2021-08-08_NDVI.tif' loaded successfully.
NDVI raster 'NW-21-002-14W1M_2021-08-13_NDVI.tif' loaded successfully.
NDVI raster 'NW-21-002-14W1M_2022-07-17_NDVI.tif' loaded successfully.
NDVI raster 'NW-21-002-14W1M_2022-08-08_NDVI.tif' loaded successfully.
NDVI raster 'NW-21-002-14W1M_2023-07-22_NDVI.tif' loaded successfully.
NDVI raster 'NW-21-002-14W1M_2024-07-28_NDVI.tif' loaded successfully.
NDVI raster 'NW-21-002-14W1M_2024-08-02_NDVI.tif' loaded success

## Reproject and align rasters

### Subtask:
Reproject and align all NDVI rasters to match the coordinate reference system and grid of the first raster.


**Reasoning**:
Implement the logic to reproject and align NDVI rasters based on the instructions provided for the current subtask.



In [6]:
import rasterio.warp

aligned_ndvi_arrays = []
target_crs = None
target_transform = None
target_width = None
target_height = None

if not original_ndvi_datasets:
    print("No NDVI datasets loaded. Skipping reprojection and alignment.")
    ndvi_datasets = original_ndvi_datasets
else:
    if skip_alignment_widget.value:
        print("Skipping reprojection and alignment as requested.")
        # If alignment is skipped, use the original datasets for subsequent steps
        ndvi_datasets = original_ndvi_datasets
        # Get target CRS and transform from the first original dataset
        if ndvi_datasets:
            target_crs = ndvi_datasets[0].crs
            target_transform = ndvi_datasets[0].transform
            target_width = ndvi_datasets[0].width
            target_height = ndvi_datasets[0].height
            print(f"Using CRS and transform from first dataset: {target_crs}")
    else:
        print("Performing reprojection and alignment...")
        # Get CRS and transform from the first dataset
        first_dataset = original_ndvi_datasets[0]
        target_crs = first_dataset.crs
        target_transform = first_dataset.transform
        target_width = first_dataset.width
        target_height = first_dataset.height

        print(f"Target CRS: {target_crs}")
        print(f"Target dimensions: {target_width} x {target_height}")

        # Read all datasets and align them
        for i, dataset in enumerate(original_ndvi_datasets):
            try:
                if i == 0:
                    # First dataset sets the reference
                    aligned_ndvi_arrays.append(dataset.read(1))
                    print(f"Dataset 0 (reference) read successfully.")
                else:
                    # Reproject other datasets to match the first
                    reprojected_data = np.empty((target_height, target_width), dtype=dataset.dtypes[0])

                    rasterio.warp.reproject(
                        source=rasterio.band(dataset, 1),
                        destination=reprojected_data,
                        src_transform=dataset.transform,
                        src_crs=dataset.crs,
                        dst_transform=target_transform,
                        dst_crs=target_crs,
                        resampling=rasterio.warp.Resampling.bilinear
                    )
                    aligned_ndvi_arrays.append(reprojected_data)
                    print(f"Dataset {i} reprojected and aligned.")
            except Exception as e:
                print(f"Error processing dataset {i}: {e}")

        # Use aligned arrays for subsequent steps
        ndvi_datasets = aligned_ndvi_arrays
        print(f"\nAlignment complete: {len(ndvi_datasets)} datasets ready for processing.")


Performing reprojection and alignment...
Target CRS: EPSG:4326
Target dimensions: 247 x 166
Dataset 0 (reference) read successfully.
Dataset 1 reprojected and aligned.
Dataset 2 reprojected and aligned.
Dataset 3 reprojected and aligned.
Dataset 4 reprojected and aligned.
Dataset 5 reprojected and aligned.
Dataset 6 reprojected and aligned.
Dataset 7 reprojected and aligned.
Dataset 8 reprojected and aligned.
Dataset 9 reprojected and aligned.
Dataset 10 reprojected and aligned.
Dataset 11 reprojected and aligned.
Dataset 12 reprojected and aligned.
Dataset 13 reprojected and aligned.

Alignment complete: 14 datasets ready for processing.


## Clip rasters to aoi

### Subtask:
Clip all aligned rasters to the boundaries of the uploaded AOI polygon.


**Reasoning**:
Check if both aoi_gdf and ndvi_datasets are available, and if so, iterate through the aligned NDVI datasets and clip each one using the aoi_gdf geometry, storing the clipped data and transforms.



In [7]:
import rasterio.mask

clipped_ndvi_data_and_transforms = []

if aoi_gdf is None or not ndvi_datasets:
    print("AOI polygon or NDVI datasets not available. Skipping clipping process.")
else:
    if target_crs is None or target_transform is None:
        print("Target CRS or Transform not defined. Cannot clip.")
    else:
        # Ensure the AOI is in the same CRS as the rasters
        if aoi_gdf.crs != target_crs:
            try:
                aoi_gdf = aoi_gdf.to_crs(target_crs)
                print(f"AOI polygon reprojected to target CRS: {target_crs}")
            except Exception as e:
                print(f"Error reprojecting AOI polygon: {e}")
                aoi_gdf = None

        if aoi_gdf is not None:
            # Get the geometry for masking
            geometries = aoi_gdf.geometry.values

            for i, dataset_or_array in enumerate(ndvi_datasets):
                try:
                    if hasattr(dataset_or_array, 'read'):
                        # It's a rasterio DatasetReader (when alignment was skipped)
                        clipped_data, clipped_transform = rasterio.mask.mask(
                            dataset_or_array, geometries, crop=True, nodata=np.nan
                        )
                        clipped_data = clipped_data[0]  # Get first band
                        clipped_ndvi_data_and_transforms.append((clipped_data, clipped_transform))
                        print(f"NDVI dataset {i} clipped successfully.")
                    else:
                        # It's a numpy array (when alignment was performed)
                        # Create a temporary in-memory dataset for clipping
                        height, width = dataset_or_array.shape
                        dtype = dataset_or_array.dtype

                        with rasterio.MemoryFile() as memfile:
                            with memfile.open(
                                driver='GTiff',
                                height=height,
                                width=width,
                                count=1,
                                dtype=dtype,
                                crs=target_crs,
                                transform=target_transform
                            ) as tmp_dataset:
                                tmp_dataset.write(dataset_or_array, 1)

                                # Perform clipping
                                clipped_data, clipped_transform = rasterio.mask.mask(
                                    tmp_dataset, geometries, crop=True, nodata=np.nan
                                )
                                clipped_data = clipped_data[0]  # Get first band

                        clipped_ndvi_data_and_transforms.append((clipped_data, clipped_transform))
                        print(f"NDVI array {i} clipped successfully.")

                except Exception as e:
                    print(f"Error clipping item {i}: {e}")
                    import traceback
                    traceback.print_exc()

            # Replace ndvi_datasets with clipped data
            ndvi_datasets = clipped_ndvi_data_and_transforms
            print(f"\nClipping complete: {len(ndvi_datasets)} datasets clipped to AOI.")


NDVI array 0 clipped successfully.
NDVI array 1 clipped successfully.
NDVI array 2 clipped successfully.
NDVI array 3 clipped successfully.
NDVI array 4 clipped successfully.
NDVI array 5 clipped successfully.
NDVI array 6 clipped successfully.
NDVI array 7 clipped successfully.
NDVI array 8 clipped successfully.
NDVI array 9 clipped successfully.
NDVI array 10 clipped successfully.
NDVI array 11 clipped successfully.
NDVI array 12 clipped successfully.
NDVI array 13 clipped successfully.

Clipping complete: 14 datasets clipped to AOI.


## Compute mean ndvi

### Subtask:
For each clipped raster, compute the mean NDVI value within the AOI.


**Reasoning**:
Check if there are clipped NDVI datasets available before attempting to compute mean values. If not, print a message and finish the task. Otherwise, iterate through the clipped datasets, calculate the mean of each, store them, and print the list of means.



In [8]:
mean_ndvi_values = []

if not ndvi_datasets:
    print("No clipped NDVI data available. Skipping mean NDVI computation.")
else:
    # Assuming ndvi_datasets now contains tuples of (clipped_numpy_array, transform)
    # from the previous clipping step.
    # If the clipping step failed and ndvi_datasets was not updated,
    # it might contain numpy arrays (the aligned data).
    # Let's assume ndvi_datasets contains numpy arrays (either clipped or aligned).
    # If it contains tuples, the code below will need adjustment to access the array part.
    # Based on the previous step's output, ndvi_datasets was replaced with clipped_ndvi_datasets
    # which are tuples.

    # Let's double-check the structure of ndvi_datasets after clipping.
    # If clipping was skipped, it might still hold the aligned numpy arrays.
    # If clipping was attempted and successful, it should hold tuples (array, transform).

    # To be safe, let's handle both cases.
    if ndvi_datasets and isinstance(ndvi_datasets[0], tuple):
        # Structure is (array, transform)
        for clipped_data, _ in ndvi_datasets:
            # Calculate the mean of the clipped data array
            mean_value = np.nanmean(clipped_data) # Use nanmean to ignore potential NoData values
            mean_ndvi_values.append(mean_value)
    elif ndvi_datasets and isinstance(ndvi_datasets[0], np.ndarray):
         # Structure is numpy array (clipping might have been skipped)
         for data_array in ndvi_datasets:
             # Calculate the mean of the data array
             mean_value = np.nanmean(data_array) # Use nanmean to ignore potential NoData values
             mean_ndvi_values.append(mean_value)
    else:
        print("Unexpected data structure in ndvi_datasets. Cannot compute mean NDVI.")


    if mean_ndvi_values:
        print("Mean NDVI values for each clipped raster:")
        print(mean_ndvi_values)
    else:
        print("Mean NDVI computation resulted in an empty list.")


Mean NDVI values for each clipped raster:
[np.float32(0.79297763), np.float32(0.7953658), np.float32(0.5171646), np.float32(0.7105119), np.float32(0.7383192), np.float32(0.78750867), np.float32(0.70209044), np.float32(0.6723136), np.float32(0.86151296), np.float32(0.84641045), np.float32(0.7684731), np.float32(0.783863), np.float32(0.8419981), np.float32(0.82733035)]


## Calculate per-year productivity index

### Subtask:
Calculate a per-year Productivity Index for each raster using the formula (NDVI / mean) * 100.


**Reasoning**:
Check if the necessary data (`ndvi_datasets` and `mean_ndvi_values`) is available and has consistent length. If so, iterate through the datasets and their corresponding mean values to calculate the per-year productivity index for each raster using the provided formula, handling potential division by zero, and store the results.



In [9]:
per_year_productivity_indices = []
mean_ndvi_values_recalc = []  # Recalculated means within AOI

if not ndvi_datasets:
    print("NDVI datasets are not available. Skipping per-year productivity index calculation.")
else:
    print(f"Processing {len(ndvi_datasets)} NDVI datasets...")
    
    # Determine structure of ndvi_datasets
    if ndvi_datasets and isinstance(ndvi_datasets[0], tuple):
        # Structure is (array, transform) from clipping step
        calculated_count = 0
        
        for i, (ndvi_data, transform) in enumerate(ndvi_datasets):
            try:
                # Calculate mean NDVI within the AOI (excluding NaN values)
                valid_mask = ~np.isnan(ndvi_data)
                
                if np.any(valid_mask):
                    mean_value = np.mean(ndvi_data[valid_mask])
                    mean_ndvi_values_recalc.append(mean_value)
                    
                    # Check if mean is too close to zero (shouldn't happen with NDVI)
                    if abs(mean_value) < 0.01:
                        print(f"Warning: Mean NDVI for raster {i} is very low ({mean_value:.4f}). This might indicate an issue with the data.")
                        # Still calculate the index
                    
                    # Calculate productivity index as ratio (not percentage)
                    # Index = NDVI / mean, where 1.0 = field average
                    productivity_index_array = np.full_like(ndvi_data, np.nan, dtype=np.float32)
                    productivity_index_array[valid_mask] = ndvi_data[valid_mask] / mean_value
                    
                    per_year_productivity_indices.append(productivity_index_array)
                    calculated_count += 1
                    print(f"  Raster {i+1}: Mean NDVI = {mean_value:.4f}, Index range = [{np.nanmin(productivity_index_array):.2f}, {np.nanmax(productivity_index_array):.2f}]")
                else:
                    print(f"  Raster {i+1}: No valid NDVI data within AOI")
                    mean_ndvi_values_recalc.append(np.nan)
                    
            except Exception as e:
                print(f"Error calculating productivity index for raster {i}: {e}")
                mean_ndvi_values_recalc.append(np.nan)
                
    elif ndvi_datasets and isinstance(ndvi_datasets[0], np.ndarray):
        # Structure is numpy array (alignment was done but no clipping)
        calculated_count = 0
        
        for i, ndvi_data in enumerate(ndvi_datasets):
            try:
                # Calculate mean NDVI (excluding NaN values)
                valid_mask = ~np.isnan(ndvi_data)
                
                if np.any(valid_mask):
                    mean_value = np.mean(ndvi_data[valid_mask])
                    mean_ndvi_values_recalc.append(mean_value)
                    
                    if abs(mean_value) < 0.01:
                        print(f"Warning: Mean NDVI for raster {i} is very low ({mean_value:.4f}).")
                    
                    # Calculate productivity index as ratio
                    productivity_index_array = np.full_like(ndvi_data, np.nan, dtype=np.float32)
                    productivity_index_array[valid_mask] = ndvi_data[valid_mask] / mean_value
                    
                    per_year_productivity_indices.append(productivity_index_array)
                    calculated_count += 1
                    print(f"  Raster {i+1}: Mean NDVI = {mean_value:.4f}, Index range = [{np.nanmin(productivity_index_array):.2f}, {np.nanmax(productivity_index_array):.2f}]")
                else:
                    print(f"  Raster {i+1}: No valid NDVI data")
                    mean_ndvi_values_recalc.append(np.nan)
                    
            except Exception as e:
                print(f"Error calculating productivity index for raster {i}: {e}")
                mean_ndvi_values_recalc.append(np.nan)
    else:
        print("Unexpected data structure in ndvi_datasets.")
    
    if per_year_productivity_indices:
        print(f"\nSuccessfully calculated {calculated_count} per-year productivity indices.")
        print(f"Productivity index values are ratios where 1.0 = field average, >1.0 = above average, <1.0 = below average")
    else:
        print("No productivity indices were calculated.")


Calculated per-year productivity indices for 14 rasters out of 14.


## Calculate multi-year composite productivity index

### Subtask:
Average the per-year Productivity Indices to create a Multi-Year Composite Productivity Index.


**Reasoning**:
Check if per-year productivity indices are available and calculate the multi-year composite index if they are.



In [10]:
composite_productivity_index = None
min_valid_observations = 2  # Require at least 2 valid observations per pixel

if not per_year_productivity_indices:
    print("No per-year productivity indices available. Skipping composite index calculation.")
else:
    print(f"Creating composite from {len(per_year_productivity_indices)} productivity indices...")
    
    # Stack all indices into a 3D array (years, height, width)
    productivity_indices_3d = np.stack(per_year_productivity_indices, axis=0)
    
    # Count valid observations per pixel
    valid_count = np.sum(~np.isnan(productivity_indices_3d), axis=0)
    
    # Calculate mean along time axis (axis=0), ignoring NaN values
    with np.errstate(invalid='ignore', divide='ignore'):
        composite_productivity_index = np.nanmean(productivity_indices_3d, axis=0)
    
    # Mask out pixels with insufficient observations
    insufficient_data_mask = valid_count < min_valid_observations
    composite_productivity_index[insufficient_data_mask] = np.nan
    
    # Calculate statistics
    valid_pixels = ~np.isnan(composite_productivity_index)
    if np.any(valid_pixels):
        composite_mean = np.mean(composite_productivity_index[valid_pixels])
        composite_std = np.std(composite_productivity_index[valid_pixels])
        composite_min = np.min(composite_productivity_index[valid_pixels])
        composite_max = np.max(composite_productivity_index[valid_pixels])
        
        print(f"Composite Productivity Index calculated successfully:")
        print(f"  Shape: {composite_productivity_index.shape}")
        print(f"  Valid pixels: {np.sum(valid_pixels)} / {composite_productivity_index.size}")
        print(f"  Mean: {composite_mean:.3f} (should be close to 1.0)")
        print(f"  Std Dev: {composite_std:.3f}")
        print(f"  Range: [{composite_min:.3f}, {composite_max:.3f}]")
        print(f"  Pixels with <{min_valid_observations} observations masked: {np.sum(insufficient_data_mask)}")
    else:
        print("Warning: No valid pixels in composite index!")


Multi-Year Composite Productivity Index calculated.
Shape of composite index: (83, 206)


/tmp/ipython-input-3660887851.py:10: RuntimeWarning: Mean of empty slice
  composite_productivity_index = np.nanmean(productivity_indices_3d, axis=0)


## Create 3-class map

### Subtask:
Create a 3-class map based on the Composite Productivity Index (below average, average, above average).


**Reasoning**:
Check if the composite productivity index is available. If it is, define the class boundaries and create the 3-class map based on these boundaries, handling potential NaN values. Finally, print the shape of the resulting map.



In [11]:
three_class_map = None
zone_thresholds = {}

if composite_productivity_index is None:
    print("Composite productivity index not available. Skipping 3-class map creation.")
else:
    print("Creating 3-zone productivity classification using quantile tertiles...")
    
    # Get valid (non-NaN) values for quantile calculation
    valid_values = composite_productivity_index[~np.isnan(composite_productivity_index)]
    
    if len(valid_values) > 0:
        # Calculate tertile thresholds (33rd and 67th percentiles)
        lower_threshold = np.percentile(valid_values, 33.33)
        upper_threshold = np.percentile(valid_values, 66.67)
        
        zone_thresholds = {
            'lower': lower_threshold,
            'upper': upper_threshold,
            'method': 'quantile_tertiles'
        }
        
        # Create the 3-class map
        three_class_map = np.full_like(composite_productivity_index, np.nan, dtype=np.float32)
        
        # Class 1: Low productivity (bottom tertile)
        three_class_map[(composite_productivity_index <= lower_threshold)] = 1
        
        # Class 2: Medium productivity (middle tertile)
        three_class_map[(composite_productivity_index > lower_threshold) & 
                       (composite_productivity_index <= upper_threshold)] = 2
        
        # Class 3: High productivity (top tertile)
        three_class_map[composite_productivity_index > upper_threshold] = 3
        
        # Calculate zone statistics
        zone_1_count = np.sum(three_class_map == 1)
        zone_2_count = np.sum(three_class_map == 2)
        zone_3_count = np.sum(three_class_map == 3)
        total_classified = zone_1_count + zone_2_count + zone_3_count
        
        print(f"3-zone classification complete:")
        print(f"  Method: Quantile tertiles (equal area distribution)")
        print(f"  Zone 1 (Low): ≤ {lower_threshold:.3f} ({zone_1_count} pixels, {zone_1_count/total_classified*100:.1f}%)")
        print(f"  Zone 2 (Medium): {lower_threshold:.3f} - {upper_threshold:.3f} ({zone_2_count} pixels, {zone_2_count/total_classified*100:.1f}%)")
        print(f"  Zone 3 (High): > {upper_threshold:.3f} ({zone_3_count} pixels, {zone_3_count/total_classified*100:.1f}%)")
        print(f"  Total classified pixels: {total_classified}")
    else:
        print("No valid values in composite index. Cannot create 3-class map.")


3-class map created.
Shape of 3-class map: (83, 206)


## Save outputs

### Subtask:
Save the per-year Productivity GeoTIFFs, the Composite GeoTIFF, the 3-class map GeoTIFF, and a CSV summary of the mean NDVI and productivity indices to a new folder in the notebook's memory.


**Reasoning**:
Create the output directory if it doesn't exist.

In [13]:
import os
import zipfile
import pandas as pd
from datetime import datetime

# Create output directory
output_dir = 'productivity_outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created output directory: {output_dir}")

# Check if required data is available
if not per_year_productivity_indices or composite_productivity_index is None or three_class_map is None:
    print("Missing required data. Please ensure all processing steps completed successfully.")
    print(f"  Per-year indices available: {len(per_year_productivity_indices) if per_year_productivity_indices else 0}")
    print(f"  Composite index available: {composite_productivity_index is not None}")
    print(f"  Three-class map available: {three_class_map is not None}")
else:
    print(f"Saving all outputs to '{output_dir}/'...")
    
    # Determine the transform and CRS to use for saving
    save_transform = None
    save_crs = None
    
    # Get transform and CRS from clipped data if available
    if ndvi_datasets and isinstance(ndvi_datasets[0], tuple):
        _, save_transform = ndvi_datasets[0]
        save_crs = target_crs if 'target_crs' in locals() else None
    elif 'target_transform' in locals() and 'target_crs' in locals():
        save_transform = target_transform
        save_crs = target_crs
    
    if save_transform is None or save_crs is None:
        print("Warning: Georeferencing information not available. Saving with default transform.")
        # Create a default transform
        height, width = composite_productivity_index.shape
        save_transform = rasterio.transform.from_bounds(0, 0, width, height, width, height)
        save_crs = 'EPSG:4326'  # Default to WGS84
    
    saved_files = []
    
    # 1. Save per-year productivity index GeoTIFFs
    for i, index_array in enumerate(per_year_productivity_indices):
        output_filename = os.path.join(output_dir, f'productivity_index_year_{i+1:02d}.tif')
        try:
            with rasterio.open(
                output_filename,
                'w',
                driver='GTiff',
                height=index_array.shape[0],
                width=index_array.shape[1],
                count=1,
                dtype='float32',
                crs=save_crs,
                transform=save_transform,
                compress='lzw',
                nodata=np.nan
            ) as dst:
                dst.write(index_array.astype(np.float32), 1)
                # Add metadata
                dst.update_tags(
                    description=f"Productivity Index Year {i+1}",
                    units="ratio",
                    interpretation="1.0 = field average, >1.0 = above average, <1.0 = below average"
                )
            saved_files.append(output_filename)
            print(f"  ✓ Year {i+1} productivity index: {output_filename}")
        except Exception as e:
            print(f"  ✗ Error saving year {i+1} index: {e}")
    
    # 2. Save composite productivity index GeoTIFF
    composite_filename = os.path.join(output_dir, 'composite_productivity_index.tif')
    try:
        with rasterio.open(
            composite_filename,
            'w',
            driver='GTiff',
            height=composite_productivity_index.shape[0],
            width=composite_productivity_index.shape[1],
            count=1,
            dtype='float32',
            crs=save_crs,
            transform=save_transform,
            compress='lzw',
            nodata=np.nan
        ) as dst:
            dst.write(composite_productivity_index.astype(np.float32), 1)
            dst.update_tags(
                description="Multi-year Composite Productivity Index",
                units="ratio",
                interpretation="1.0 = field average, >1.0 = above average, <1.0 = below average",
                years_averaged=str(len(per_year_productivity_indices))
            )
        saved_files.append(composite_filename)
        print(f"  ✓ Composite productivity index: {composite_filename}")
    except Exception as e:
        print(f"  ✗ Error saving composite index: {e}")
    
    # 3. Save 3-zone classification GeoTIFF with color table
    zones_filename = os.path.join(output_dir, 'productivity_zones.tif')
    try:
        with rasterio.open(
            zones_filename,
            'w',
            driver='GTiff',
            height=three_class_map.shape[0],
            width=three_class_map.shape[1],
            count=1,
            dtype='int16',
            crs=save_crs,
            transform=save_transform,
            compress='lzw',
            nodata=-9999
        ) as dst:
            # Convert NaN to nodata value for integer raster
            zones_int = three_class_map.copy()
            zones_int[np.isnan(zones_int)] = -9999
            dst.write(zones_int.astype(np.int16), 1)
            
            # Add color interpretation (red-yellow-green for low-medium-high)
            dst.write_colormap(1, {
                1: (255, 67, 67, 255),    # Red for low productivity
                2: (255, 255, 102, 255),  # Yellow for medium
                3: (76, 230, 0, 255),     # Green for high
                -9999: (0, 0, 0, 0)       # Transparent for nodata
            })
            
            # Add metadata
            if 'zone_thresholds' in locals() and zone_thresholds:
                dst.update_tags(
                    description="Productivity Zones (3-class)",
                    classification_method=zone_thresholds.get('method', 'quantile_tertiles'),
                    zone_1="Low productivity",
                    zone_2="Medium productivity", 
                    zone_3="High productivity",
                    lower_threshold=str(zone_thresholds.get('lower', '')),
                    upper_threshold=str(zone_thresholds.get('upper', ''))
                )
        saved_files.append(zones_filename)
        print(f"  ✓ Productivity zones: {zones_filename}")
    except Exception as e:
        print(f"  ✗ Error saving zones: {e}")
    
    # 4. Create summary statistics CSV
    summary_filename = os.path.join(output_dir, 'productivity_summary.csv')
    try:
        summary_data = []
        
        # Per-year statistics
        for i, (mean_ndvi, index_array) in enumerate(zip(mean_ndvi_values_recalc, per_year_productivity_indices)):
            valid_index = index_array[~np.isnan(index_array)]
            if len(valid_index) > 0:
                summary_data.append({
                    'Year': i + 1,
                    'Mean_NDVI': mean_ndvi if not np.isnan(mean_ndvi) else None,
                    'Index_Mean': np.mean(valid_index),
                    'Index_StdDev': np.std(valid_index),
                    'Index_Min': np.min(valid_index),
                    'Index_Max': np.max(valid_index),
                    'Valid_Pixels': len(valid_index)
                })
        
        # Composite statistics
        valid_composite = composite_productivity_index[~np.isnan(composite_productivity_index)]
        if len(valid_composite) > 0:
            summary_data.append({
                'Year': 'Composite',
                'Mean_NDVI': None,
                'Index_Mean': np.mean(valid_composite),
                'Index_StdDev': np.std(valid_composite),
                'Index_Min': np.min(valid_composite),
                'Index_Max': np.max(valid_composite),
                'Valid_Pixels': len(valid_composite)
            })
        
        # Zone thresholds
        if 'zone_thresholds' in locals() and zone_thresholds:
            summary_data.append({
                'Year': 'Zone_Thresholds',
                'Mean_NDVI': None,
                'Index_Mean': zone_thresholds.get('lower'),
                'Index_StdDev': zone_thresholds.get('upper'),
                'Index_Min': None,
                'Index_Max': None,
                'Valid_Pixels': None
            })
        
        # Save to CSV
        df_summary = pd.DataFrame(summary_data)
        df_summary.to_csv(summary_filename, index=False)
        saved_files.append(summary_filename)
        print(f"  ✓ Summary statistics: {summary_filename}")
    except Exception as e:
        print(f"  ✗ Error saving summary CSV: {e}")
    
    # 5. Create ZIP archive for download
    zip_filename = 'productivity_outputs.zip'
    try:
        with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for file in saved_files:
                if os.path.exists(file):
                    zipf.write(file, os.path.basename(file))
        print(f"\n✓ Created ZIP archive: {zip_filename}")
        
        # Provide download mechanism based on environment
        if IN_COLAB:
            from google.colab import files
            files.download(zip_filename)
            print("Download initiated via Colab...")
        else:
            print(f"Download the outputs from: {os.path.abspath(zip_filename)}")
            
    except Exception as e:
        print(f"✗ Error creating ZIP archive: {e}")
    
    print(f"\nProcessing complete! {len(saved_files)} files saved.")
    print("Files include:")
    print(f"  - {len(per_year_productivity_indices)} per-year productivity index rasters")
    print(f"  - 1 composite productivity index raster")
    print(f"  - 1 productivity zones raster (3 classes)")
    print(f"  - 1 summary statistics CSV")


Created output directory: productivity_outputs


**Reasoning**:
Check for the availability of required data and then proceed to save the results to GeoTIFF and CSV files in the output directory.



In [14]:
import pandas as pd

# Check if required data is available
if not per_year_productivity_indices or composite_productivity_index is None or three_class_map is None or not mean_ndvi_values or not ndvi_datasets:
    print("Required data (per-year productivity indices, composite index, 3-class map, mean NDVI, or original NDVI datasets) is not fully available. Skipping saving.")
else:
    print(f"Saving output files to '{output_dir}'...")

    # Assuming the structure of ndvi_datasets is now (clipped_array, transform) or just numpy arrays
    # We need the original transforms and CRSs for saving.
    # If clipping was successful, the transform is within the tuple.
    # If clipping was skipped, ndvi_datasets still holds the aligned numpy arrays.
    # We need to access the original rasterio datasets to get the CRS and original transform
    # before clipping and alignment.
    # Assuming original_ndvi_datasets (list of rasterio dataset objects) is available
    # from the loading step. This was not explicitly preserved.
    # Let's assume we stored the target_transform and target_crs from the alignment step.

    try:
        target_transform
        target_crs
    except NameError:
        print("Target transform or CRS not defined from previous steps. Cannot save GeoTIFFs.")
    else:
        # Save per-year Productivity GeoTIFFs
        for i, per_year_index_array in enumerate(per_year_productivity_indices):
            # Use the target transform and CRS for saving
            output_filename = os.path.join(output_dir, f'per_year_productivity_{i+1}.tif')
            try:
                height, width = per_year_index_array.shape
                dtype = per_year_index_array.dtype
                with rasterio.open(
                    output_filename,
                    'w',
                    driver='GTiff',
                    height=height,
                    width=width,
                    count=1,
                    dtype=dtype,
                    crs=target_crs,
                    transform=target_transform,
                    nodata=np.nan # Specify NoData value
                ) as dst:
                    dst.write(per_year_index_array, 1)
                print(f"Saved per-year productivity GeoTIFF: {output_filename}")
            except Exception as e:
                print(f"Error saving per-year productivity GeoTIFF {output_filename}: {e}")

        # Save Composite GeoTIFF
        output_composite_filename = os.path.join(output_dir, 'composite_productivity.tif')
        try:
            height, width = composite_productivity_index.shape
            dtype = composite_productivity_index.dtype
            with rasterio.open(
                output_composite_filename,
                'w',
                driver='GTiff',
                height=height,
                width=width,
                count=1,
                dtype=dtype,
                crs=target_crs,
                transform=target_transform,
                nodata=np.nan # Specify NoData value
            ) as dst:
                dst.write(composite_productivity_index, 1)
            print(f"Saved Composite Productivity GeoTIFF: {output_composite_filename}")
        except Exception as e:
            print(f"Error saving Composite Productivity GeoTIFF {output_composite_filename}: {e}")

        # Save 3-class map GeoTIFF
        output_map_filename = os.path.join(output_dir, 'three_class_map.tif')
        try:
            height, width = three_class_map.shape
            # The data type of the 3-class map is likely integer or float,
            # let's ensure it's an appropriate type for GeoTIFF.
            # Using float32 for consistency, but integer type might be better
            # depending on the required output.
            dtype = np.float32
            with rasterio.open(
                output_map_filename,
                'w',
                driver='GTiff',
                height=height,
                width=width,
                count=1,
                dtype=dtype,
                crs=target_crs,
                transform=target_transform,
                nodata=np.nan # Use NaN or a specific integer for NoData if using integer dtype
            ) as dst:
                dst.write(three_class_map.astype(dtype), 1) # Ensure correct dtype is written
            print(f"Saved 3-class map GeoTIFF: {output_map_filename}")
        except Exception as e:
            print(f"Error saving 3-class map GeoTIFF {output_map_filename}: {e}")

    # Create and save CSV summary
    # Assuming we have filenames available from the upload step (ndvi_uploads.value.keys())
    # and corresponding mean_ndvi_values and potentially mean productivity indices.
    # We don't have mean productivity indices calculated explicitly.
    # Let's create a DataFrame with filenames (if available) and mean_ndvi_values.

    summary_data = {'Mean_NDVI': mean_ndvi_values}

    # Try to get original filenames if ndvi_uploads is available and populated
    if 'ndvi_uploads' in locals() and ndvi_uploads.value:
         # Get filenames in the order they were likely processed
         # Assuming the order in ndvi_uploads.value.keys() matches the order in mean_ndvi_values
         filenames = list(ndvi_uploads.value.keys())[:len(mean_ndvi_values)]
         summary_data['Original_Filename'] = filenames
         # Reorder columns
         summary_df = pd.DataFrame(summary_data, columns=['Original_Filename', 'Mean_NDVI'])
    else:
        summary_df = pd.DataFrame(summary_data)


    output_csv_filename = os.path.join(output_dir, 'productivity_summary.csv')
    try:
        summary_df.to_csv(output_csv_filename, index=False)
        print(f"Saved CSV summary: {output_csv_filename}")
    except Exception as e:
        print(f"Error saving CSV summary {output_csv_filename}: {e}")


Saving output files to 'productivity_outputs'...
Saved per-year productivity GeoTIFF: productivity_outputs/per_year_productivity_1.tif
Saved per-year productivity GeoTIFF: productivity_outputs/per_year_productivity_2.tif
Saved per-year productivity GeoTIFF: productivity_outputs/per_year_productivity_3.tif
Saved per-year productivity GeoTIFF: productivity_outputs/per_year_productivity_4.tif
Saved per-year productivity GeoTIFF: productivity_outputs/per_year_productivity_5.tif
Saved per-year productivity GeoTIFF: productivity_outputs/per_year_productivity_6.tif
Saved per-year productivity GeoTIFF: productivity_outputs/per_year_productivity_7.tif
Saved per-year productivity GeoTIFF: productivity_outputs/per_year_productivity_8.tif
Saved per-year productivity GeoTIFF: productivity_outputs/per_year_productivity_9.tif
Saved per-year productivity GeoTIFF: productivity_outputs/per_year_productivity_10.tif
Saved per-year productivity GeoTIFF: productivity_outputs/per_year_productivity_11.tif
Sav

## Download Outputs

### Subtask:
Zip the output folder and provide a download link.

In [15]:
import shutil
from google.colab import files

# Define the output directory
output_dir = 'productivity_outputs'
zip_filename = f'{output_dir}.zip'

# Create a zip archive of the output directory
try:
    shutil.make_archive(output_dir, 'zip', output_dir)
    print(f"Created zip archive: {zip_filename}")

    # Provide a download link in Colab
    if IN_COLAB:
        print("Click the link below to download the zip file:")
        files.download(zip_filename)
    else:
        print(f"Zip file created at: {zip_filename}. You can download it manually.")

except FileNotFoundError:
    print(f"Error: Output directory '{output_dir}' not found. Please ensure the saving step was successful.")
except Exception as e:
    print(f"Error zipping and downloading files: {e}")

Created zip archive: productivity_outputs.zip
Click the link below to download the zip file:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Task
Analyze the relationship between the calculated productivity indices and the provided DSM and TPI raster data.

## Upload dsm and tpi data

### Subtask:
Add cells to upload the DSM and TPI raster files.


**Reasoning**:
Create and display the file upload widgets for DSM and TPI rasters as per the instructions.



In [16]:
# Create file upload widgets for DSM and TPI
dsm_uploads = widgets.FileUpload(
    accept='.tif,.tiff',
    multiple=True,
    description='Upload DSM Rasters (GeoTIFFs)'
)

tpi_uploads = widgets.FileUpload(
    accept='.tif,.tiff',
    multiple=True,
    description='Upload TPI Rasters (GeoTIFFs)'
)

display(dsm_uploads)
display(tpi_uploads)
print("File upload widgets created for DSM and TPI. Please upload the respective files.")

FileUpload(value={}, accept='.tif,.tiff', description='Upload DSM Rasters (GeoTIFFs)', multiple=True)

FileUpload(value={}, accept='.tif,.tiff', description='Upload TPI Rasters (GeoTIFFs)', multiple=True)

File upload widgets created for DSM and TPI. Please upload the respective files.


## Load dsm and tpi data

### Subtask:
Load the uploaded DSM and TPI rasters into rasterio datasets.


**Reasoning**:
Check for uploaded DSM and TPI files and load them into rasterio datasets using MemoryFile, then store the datasets and initialize lists for later use.



In [17]:
original_dsm_datasets = []
original_tpi_datasets = []
dsm_memfiles = [] # Keep MemoryFile objects alive for DSM
tpi_memfiles = [] # Keep MemoryFile objects alive for TPI

# Load DSM rasters
dsm_files = get_file_content(dsm_uploads, is_multiple=True)
if dsm_files:
    for file_data in dsm_files:
        try:
            memfile = rasterio.io.MemoryFile(file_data['content'])
            dataset = memfile.open()
            original_dsm_datasets.append(dataset)
            dsm_memfiles.append(memfile) # Important: Keep memfile alive!
            print(f"DSM raster '{file_data['name']}' loaded successfully.")
        except Exception as e:
            print(f"Error loading DSM file '{file_data['name']}': {e}")
else:
    print("No DSM files uploaded. Skipping DSM loading.")

# Load TPI rasters
tpi_files = get_file_content(tpi_uploads, is_multiple=True)
if tpi_files:
    for file_data in tpi_files:
        try:
            memfile = rasterio.io.MemoryFile(file_data['content'])
            dataset = memfile.open()
            original_tpi_datasets.append(dataset)
            tpi_memfiles.append(memfile) # Important: Keep memfile alive!
            print(f"TPI raster '{file_data['name']}' loaded successfully.")
        except Exception as e:
            print(f"Error loading TPI file '{file_data['name']}': {e}")
else:
    print("No TPI files uploaded. Skipping TPI loading.")

# Initialize dsm_datasets and tpi_datasets for later use
dsm_datasets = []
tpi_datasets = []

# Summary
print(f"\nSummary: Loaded {len(original_dsm_datasets)} DSM raster(s) and {len(original_tpi_datasets)} TPI raster(s).")


No DSM files uploaded. Skipping DSM loading.
No TPI files uploaded. Skipping TPI loading.

Summary: Loaded 0 DSM raster(s) and 0 TPI raster(s).


## Reproject and align dsm and tpi

### Subtask:
Reproject and align the DSM and TPI rasters to match the grid and CRS of the NDVI data (or a common reference).


**Reasoning**:
Check for the availability of DSM and TPI datasets and the target reference variables, then reproject and align the available datasets to the target grid and CRS, storing the results in new lists.



In [18]:
aligned_dsm_arrays = []
aligned_tpi_arrays = []

# 1. Check if there are any original_dsm_datasets or original_tpi_datasets to process.
if not original_dsm_datasets and not original_tpi_datasets:
    print("No original DSM or TPI datasets loaded. Skipping reprojection and alignment for DSM/TPI.")
else:
    # 2. Determine the target coordinate reference system (CRS), transform, width, and height.
    try:
        target_crs
        target_transform
        target_width
        target_height
        print(f"Using target CRS: {target_crs}, transform, width: {target_width}, height: {target_height} from NDVI alignment.")
    except NameError:
        print("Target CRS, transform, width, or height from NDVI alignment are not available. Cannot reproject/align DSM/TPI.")
    else:
        # 4. Iterate through the original_dsm_datasets (if any) and reproject/align them.
        if original_dsm_datasets:
            print("Processing DSM datasets for reprojection and alignment...")
            for i, dataset in enumerate(original_dsm_datasets):
                try:
                    reprojected_data = np.empty((target_height, target_width), dtype=dataset.dtypes[0])

                    rasterio.warp.reproject(
                        source=rasterio.band(dataset, 1),
                        destination=reprojected_data,
                        src_transform=dataset.transform,
                        src_crs=dataset.crs,
                        dst_transform=target_transform,
                        dst_crs=target_crs,
                        resampling=rasterio.warp.Resampling.bilinear,
                        dst_nodata=dataset.nodata # Use source nodata for destination
                    )
                    aligned_dsm_arrays.append(reprojected_data)
                    print(f"DSM dataset {i} reprojected and aligned.")
                except Exception as e:
                    print(f"Error processing DSM dataset {i}: {e}")
                    import traceback
                    traceback.print_exc()

        # 5. Iterate through the original_tpi_datasets (if any) and reproject/align them.
        if original_tpi_datasets:
            print("Processing TPI datasets for reprojection and alignment...")
            for i, dataset in enumerate(original_tpi_datasets):
                try:
                    reprojected_data = np.empty((target_height, target_width), dtype=dataset.dtypes[0])

                    rasterio.warp.reproject(
                        source=rasterio.band(dataset, 1),
                        destination=reprojected_data,
                        src_transform=dataset.transform,
                        src_crs=dataset.crs,
                        dst_transform=target_transform,
                        dst_crs=target_crs,
                        resampling=rasterio.warp.Resampling.bilinear,
                        dst_nodata=dataset.nodata # Use source nodata for destination
                    )
                    aligned_tpi_arrays.append(reprojected_data)
                    print(f"TPI dataset {i} reprojected and aligned.")
                except Exception as e:
                    print(f"Error processing TPI dataset {i}: {e}")
                    import traceback
                    traceback.print_exc()

        # 6. After processing, update dsm_datasets with aligned_dsm_arrays and tpi_datasets with aligned_tpi_arrays.
        dsm_datasets = aligned_dsm_arrays
        tpi_datasets = aligned_tpi_arrays

        # 7. Print a summary message
        print(f"\nReprojection and alignment complete: {len(dsm_datasets)} DSM and {len(tpi_datasets)} TPI datasets are ready.")


No original DSM or TPI datasets loaded. Skipping reprojection and alignment for DSM/TPI.


## Clip dsm and tpi to aoi

### Subtask:
Clip the aligned DSM and TPI rasters to the AOI polygon.


**Reasoning**:
Check for the availability of AOI, DSM, and TPI data, then iterate through DSM and TPI datasets (if available), create temporary in-memory datasets with the target transform and CRS, clip them using the AOI geometry, store the clipped data and transforms, update the dsm_datasets and tpi_datasets lists, and print a summary.



In [19]:
import rasterio.mask

clipped_dsm_data_and_transforms = []
clipped_tpi_data_and_transforms = []

# 1. Check if both aoi_gdf and either dsm_datasets or tpi_datasets are available.
if aoi_gdf is None or (not dsm_datasets and not tpi_datasets):
    print("AOI polygon, DSM datasets, or TPI datasets not available. Skipping clipping process for DSM/TPI.")
else:
    # 2. Ensure the AOI is in the same CRS as the target CRS used for alignment (target_crs).
    try:
        target_crs
        target_transform
    except NameError:
        print("Target CRS or Transform not defined from previous steps. Cannot clip DSM/TPI.")
    else:
        if aoi_gdf.crs != target_crs:
            try:
                aoi_gdf = aoi_gdf.to_crs(target_crs)
                print(f"AOI polygon reprojected to target CRS: {target_crs}")
            except Exception as e:
                print(f"Error reprojecting AOI polygon: {e}")
                aoi_gdf = None # Set to None if reprojection fails

        if aoi_gdf is not None:
            # 3. Get the geometry of the aoi_gdf for masking.
            geometries = aoi_gdf.geometry.values

            # 4. If dsm_datasets are available, iterate and clip them.
            if dsm_datasets:
                print("Clipping DSM datasets to AOI...")
                for i, dsm_array in enumerate(dsm_datasets):
                    try:
                        # Create a temporary in-memory dataset for clipping
                        height, width = dsm_array.shape
                        dtype = dsm_array.dtype

                        with rasterio.MemoryFile() as memfile:
                            with memfile.open(
                                driver='GTiff',
                                height=height,
                                width=width,
                                count=1,
                                dtype=dtype,
                                crs=target_crs,
                                transform=target_transform
                            ) as tmp_dataset:
                                tmp_dataset.write(dsm_array, 1)

                                # Perform clipping
                                clipped_data, clipped_transform = rasterio.mask.mask(
                                    tmp_dataset, geometries, crop=True, nodata=np.nan # Use np.nan for NoData
                                )
                                clipped_data = clipped_data[0]  # Get first band

                        clipped_dsm_data_and_transforms.append((clipped_data, clipped_transform))
                        print(f"DSM dataset {i} clipped successfully.")

                    except Exception as e:
                        print(f"Error clipping DSM dataset {i}: {e}")
                        import traceback
                        traceback.print_exc()

            # 5. If tpi_datasets are available, iterate and clip them.
            if tpi_datasets:
                print("Clipping TPI datasets to AOI...")
                for i, tpi_array in enumerate(tpi_datasets):
                    try:
                        # Create a temporary in-memory dataset for clipping
                        height, width = tpi_array.shape
                        dtype = tpi_array.dtype

                        with rasterio.MemoryFile() as memfile:
                            with memfile.open(
                                driver='GTiff',
                                height=height,
                                width=width,
                                count=1,
                                dtype=dtype,
                                crs=target_crs,
                                transform=target_transform
                            ) as tmp_dataset:
                                tmp_dataset.write(tpi_array, 1)

                                # Perform clipping
                                clipped_data, clipped_transform = rasterio.mask.mask(
                                    tmp_dataset, geometries, crop=True, nodata=np.nan # Use np.nan for NoData
                                )
                                clipped_data = clipped_data[0]  # Get first band

                        clipped_tpi_data_and_transforms.append((clipped_data, clipped_transform))
                        print(f"TPI dataset {i} clipped successfully.")

                    except Exception as e:
                        print(f"Error clipping TPI dataset {i}: {e}")
                        import traceback
                        traceback.print_exc()

        # 6. After processing, update dsm_datasets and tpi_datasets.
        dsm_datasets = clipped_dsm_data_and_transforms
        tpi_datasets = clipped_tpi_data_and_transforms

        # 7. Print a summary message
        print(f"\nClipping complete: {len(dsm_datasets)} DSM and {len(tpi_datasets)} TPI datasets clipped to AOI.")


AOI polygon, DSM datasets, or TPI datasets not available. Skipping clipping process for DSM/TPI.


## Analyze relationship between productivity and topography

### Subtask:
Explore the relationship between the calculated productivity indices (per-year or composite) and the DSM and TPI values within the AOI. This could involve:
    - Zonal statistics: Calculate mean or median DSM/TPI within each productivity class.
    - Scatter plots: Plot productivity values against DSM or TPI values for individual pixels.
    - Spatial analysis: Identify areas of high/low productivity that correlate with specific topographic features (e.g., ridges, valleys).


**Reasoning**:
Check for the availability of required data and if available, extract, flatten, and create a DataFrame from the composite productivity index, clipped DSM, and clipped TPI data. Then, calculate and print the correlation matrix.



In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Check if required data is available
if composite_productivity_index is None or not clipped_dsm_data_and_transforms or not clipped_tpi_data_and_transforms:
    print("Required data (composite productivity index, clipped DSM, or clipped TPI) is not available. Skipping relationship analysis.")
else:
    print("Analyzing relationship between productivity index, DSM, and TPI...")

    # 2. Extract the clipped DSM and TPI arrays
    # Assuming there is only one DSM and one TPI raster
    clipped_dsm_array, _ = clipped_dsm_data_and_transforms[0]
    clipped_tpi_array, _ = clipped_tpi_data_and_transforms[0]

    # Ensure the arrays have the same shape as the composite index
    if composite_productivity_index.shape != clipped_dsm_array.shape or composite_productivity_index.shape != clipped_tpi_array.shape:
        print("Error: Composite productivity index, DSM, and TPI arrays do not have consistent shapes. Cannot proceed with analysis.")
    else:
        # 3. Flatten the arrays, ignoring NaN values
        composite_flat = composite_productivity_index[~np.isnan(composite_productivity_index)].flatten()
        dsm_flat = clipped_dsm_array[~np.isnan(composite_productivity_index)].flatten() # Use composite index mask to align
        tpi_flat = clipped_tpi_array[~np.isnan(composite_productivity_index)].flatten() # Use composite index mask to align

        # Remove NaN values from the flattened arrays to ensure they have the same length
        # Create a combined mask for NaN values across all arrays
        combined_nan_mask = np.isnan(composite_flat) | np.isnan(dsm_flat) | np.isnan(tpi_flat)

        composite_flat_cleaned = composite_flat[~combined_nan_mask]
        dsm_flat_cleaned = dsm_flat[~combined_nan_mask]
        tpi_flat_cleaned = tpi_flat[~combined_nan_mask]


        if composite_flat_cleaned.size == 0:
             print("No valid data points available after removing NaN values. Skipping analysis.")
        else:
            # 4. Create a pandas DataFrame
            analysis_df = pd.DataFrame({
                'Composite_Productivity': composite_flat_cleaned,
                'DSM': dsm_flat_cleaned,
                'TPI': tpi_flat_cleaned
            })

            # 5. Calculate and print the correlation matrix
            correlation_matrix = analysis_df.corr()
            print("\nCorrelation Matrix:")
            display(correlation_matrix)

            # 6. Generate scatter plots
            print("\nGenerating Scatter Plots:")

            plt.figure(figsize=(12, 5))

            plt.subplot(1, 2, 1)
            plt.scatter(analysis_df['DSM'], analysis_df['Composite_Productivity'], alpha=0.5, s=10)
            plt.title('Composite Productivity vs. DSM')
            plt.xlabel('DSM')
            plt.ylabel('Composite Productivity Index')
            plt.grid(True)

            plt.subplot(1, 2, 2)
            plt.scatter(analysis_df['TPI'], analysis_df['Composite_Productivity'], alpha=0.5, s=10, color='orange')
            plt.title('Composite Productivity vs. TPI')
            plt.xlabel('TPI')
            plt.ylabel('Composite Productivity Index')
            plt.grid(True)

            plt.tight_layout()
            plt.show()

            # 7. Print a summary of the correlation analysis
            print("\nCorrelation Analysis Summary:")
            print("The correlation matrix shows the linear relationship between the variables.")
            print(f"- Correlation between Composite Productivity and DSM: {correlation_matrix.loc['Composite_Productivity', 'DSM']:.2f}")
            print(f"- Correlation between Composite Productivity and TPI: {correlation_matrix.loc['Composite_Productivity', 'TPI']:.2f}")
            print("Scatter plots visualize these relationships.")
            print("A correlation value close to 1 indicates a strong positive linear relationship, close to -1 indicates a strong negative linear relationship, and close to 0 indicates a weak linear relationship.")


Required data (composite productivity index, clipped DSM, or clipped TPI) is not available. Skipping relationship analysis.


## Analyze relationship between productivity and topography - retry 1

### Subtask:
Retry: Explore the relationship between the calculated productivity indices (per-year or composite) and the DSM and TPI values within the AOI. This could involve:
    - Zonal statistics: Calculate mean or median DSM/TPI within each productivity class.
    - Scatter plots: Plot productivity values against DSM or TPI values for individual pixels.
    - Spatial analysis: Identify areas of high/low productivity that correlate with specific topographic features (e.g., ridges, valleys).


**Reasoning**:
Check for the availability of required data and then proceed to perform the relationship analysis, including creating a DataFrame, calculating the correlation matrix, generating scatter plots, performing zonal statistics based on the 3-class map, and finally summarizing the findings.



In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio.features

# 1. Check if required data is available
# Ensure all necessary variables are available before proceeding
required_vars = ['composite_productivity_index', 'three_class_map', 'clipped_dsm_data_and_transforms', 'clipped_tpi_data_and_transforms', 'target_transform']
if not all(var in locals() and locals()[var] is not None for var in ['composite_productivity_index', 'three_class_map']) or \
   not clipped_dsm_data_and_transforms or not clipped_tpi_data_and_transforms or \
   'target_transform' not in locals():
    print("Required data (composite productivity index, 3-class map, clipped DSM, clipped TPI, or target transform) is not fully available. Skipping relationship analysis.")
else:
    print("Analyzing relationship between productivity index, DSM, and TPI...")

    # 2. Extract the first clipped DSM and TPI arrays and their transforms
    # Assuming there is only one DSM and one TPI raster
    clipped_dsm_array, clipped_dsm_transform = clipped_dsm_data_and_transforms[0]
    clipped_tpi_array, clipped_tpi_transform = clipped_tpi_data_and_transforms[0]

    # Ensure the arrays have consistent shapes
    if composite_productivity_index.shape != clipped_dsm_array.shape or composite_productivity_index.shape != clipped_tpi_array.shape or composite_productivity_index.shape != three_class_map.shape:
        print(f"Error: Array shapes are inconsistent:")
        print(f"  Composite Productivity: {composite_productivity_index.shape}")
        print(f"  Clipped DSM: {clipped_dsm_array.shape}")
        print(f"  Clipped TPI: {clipped_tpi_array.shape}")
        print(f"  3-Class Map: {three_class_map.shape}")
        print("Cannot proceed with analysis.")
    else:
        # 3. Flatten the arrays, handling NaN values consistently
        # Create a mask for valid data points (where composite_productivity_index is not NaN)
        valid_mask = ~np.isnan(composite_productivity_index)

        composite_flat_masked = composite_productivity_index[valid_mask].flatten()
        dsm_flat_masked = clipped_dsm_array[valid_mask].flatten()
        tpi_flat_masked = clipped_tpi_array[valid_mask].flatten()
        three_class_flat_masked = three_class_map[valid_mask].flatten()


        # Further remove NaNs that might be present in DSM or TPI within the valid productivity areas
        combined_nan_mask = np.isnan(composite_flat_masked) | np.isnan(dsm_flat_masked) | np.isnan(tpi_flat_masked) | np.isnan(three_class_flat_masked)


        composite_flat_cleaned = composite_flat_masked[~combined_nan_mask]
        dsm_flat_cleaned = dsm_flat_masked[~combined_nan_mask]
        tpi_flat_cleaned = tpi_flat_masked[~combined_nan_mask]
        three_class_flat_cleaned = three_class_flat_masked[~combined_nan_mask]


        # 5. Check if there are valid data points left
        if composite_flat_cleaned.size == 0:
             print("No valid data points available after removing NaN values. Skipping analysis.")
        else:
            # 6. Create a pandas DataFrame for correlation and scatter plots
            analysis_df = pd.DataFrame({
                'Composite_Productivity': composite_flat_cleaned,
                'DSM': dsm_flat_cleaned,
                'TPI': tpi_flat_cleaned,
                'Productivity_Class': three_class_flat_cleaned # Include class for zonal stats
            })

            # 7. Calculate and print the correlation matrix
            print("\nCorrelation Matrix:")
            # Calculate correlation only for the numeric columns
            correlation_matrix = analysis_df[['Composite_Productivity', 'DSM', 'TPI']].corr()
            display(correlation_matrix)

            # 8. Generate scatter plots
            print("\nGenerating Scatter Plots:")

            plt.figure(figsize=(12, 5))

            plt.subplot(1, 2, 1)
            plt.scatter(analysis_df['DSM'], analysis_df['Composite_Productivity'], alpha=0.5, s=5)
            plt.title('Composite Productivity vs. DSM')
            plt.xlabel('DSM')
            plt.ylabel('Composite Productivity Index')
            plt.grid(True)

            plt.subplot(1, 2, 2)
            plt.scatter(analysis_df['TPI'], analysis_df['Composite_Productivity'], alpha=0.5, s=5, color='orange')
            plt.title('Composite Productivity vs. TPI')
            plt.xlabel('TPI')
            plt.ylabel('Composite Productivity Index')
            plt.grid(True)

            plt.tight_layout()
            plt.show()

            # 9. Perform basic zonal statistics analysis
            print("\nZonal Statistics (Mean DSM and TPI by Productivity Class):")
            zonal_stats = analysis_df.groupby('Productivity_Class')[['DSM', 'TPI']].mean().reset_index()

            # Map class numbers to descriptions for readability
            class_mapping = {1: 'Below Average', 2: 'Average', 3: 'Above Average'}
            zonal_stats['Productivity_Class'] = zonal_stats['Productivity_Class'].map(class_mapping)

            display(zonal_stats)

            # 10. Print a summary of the findings
            print("\nSummary of Findings:")
            print("Based on the analysis:")
            print("- Correlation Analysis:")
            print(f"  - Composite Productivity has a correlation of {correlation_matrix.loc['Composite_Productivity', 'DSM']:.2f} with DSM.")
            print(f"  - Composite Productivity has a correlation of {correlation_matrix.loc['Composite_Productivity', 'TPI']:.2f} with TPI.")
            print("  (Note: Correlation indicates the strength and direction of a linear relationship.)")
            print("- Zonal Statistics:")
            print("  - The mean DSM and TPI values vary across different productivity classes.")
            for index, row in zonal_stats.iterrows():
                print(f"  - For '{row['Productivity_Class']}' productivity, the mean DSM is {row['DSM']:.2f} and the mean TPI is {row['TPI']:.2f}.")

            # Basic interpretation based on common relationships (can be expanded)
            print("\nPotential Interpretation:")
            if correlation_matrix.loc['Composite_Productivity', 'DSM'] > 0:
                print("- Higher DSM values (generally higher elevations) tend to be associated with higher composite productivity.")
            elif correlation_matrix.loc['Composite_Productivity', 'DSM'] < 0:
                print("- Lower DSM values (generally lower elevations) tend to be associated with higher composite productivity.")
            else:
                print("- Little linear relationship observed between Composite Productivity and DSM.")

            if correlation_matrix.loc['Composite_Productivity', 'TPI'] > 0:
                print("- Higher TPI values (generally ridges or convex areas) tend to be associated with higher composite productivity.")
            elif correlation_matrix.loc['Composite_Productivity', 'TPI'] < 0:
                print("- Lower TPI values (generally valleys or concave areas) tend to be associated with higher composite productivity.")
            else:
                print("- Little linear relationship observed between Composite Productivity and TPI.")

            # Additional interpretation based on zonal stats
            print("\nConsider the zonal statistics to understand the average topographic characteristics within each productivity class.")

Required data (composite productivity index, 3-class map, clipped DSM, clipped TPI, or target transform) is not fully available. Skipping relationship analysis.


## Visualize combined data

### Subtask:
Create visualizations that combine the productivity maps with the topographic data (e.g., overlaying productivity classes on a hillshade of the DSM).


**Reasoning**:
Check for the availability of required data and then proceed to create the visualization by generating a hillshade from the DSM, defining a colormap for the 3-class map, and overlaying the 3-class map on the hillshade.



In [22]:
import rasterio.plot
import matplotlib.colors as mcolors

# 1. Check if required data is available
if three_class_map is None or not clipped_dsm_data_and_transforms or 'target_transform' not in locals():
    print("Required data (3-class map, clipped DSM, or target transform) is not fully available. Skipping visualization.")
else:
    print("Creating visualization combining productivity map and DSM hillshade...")

    # 2. Extract the clipped DSM array and its transform
    # Assuming there is only one clipped DSM raster
    clipped_dsm_array, clipped_dsm_transform = clipped_dsm_data_and_transforms[0]

    # Ensure the 3-class map and DSM array have consistent shapes
    if three_class_map.shape != clipped_dsm_array.shape:
        print(f"Error: 3-class map shape ({three_class_map.shape}) and clipped DSM shape ({clipped_dsm_array.shape}) are inconsistent. Cannot create visualization.")
    else:
        # 3. Calculate a hillshade from the clipped DSM array
        try:
            # Using rasterio's hillshade function
            hillshade_array = rasterio.plot.make_hillshade(clipped_dsm_array, dx=clipped_dsm_transform[0], dy=clipped_dsm_transform[4])
            print("Hillshade calculated from DSM.")
        except Exception as e:
            print(f"Error calculating hillshade: {e}")
            hillshade_array = None # Set to None if hillshade calculation fails

        if hillshade_array is not None:
            # 4. Define a colormap and normalization for the 3-class map
            # Define colors for each class (e.g., red for below, yellow for average, green for above)
            colors = ['#E41A1C', '#FFC000', '#4DAF4A'] # Red, Amber, Green (colorblind-friendly palette)
            class_labels = ['Below Average', 'Average', 'Above Average']
            class_values = [1, 2, 3]

            cmap = mcolors.ListedColormap(colors)
            # Define boundaries for the normalization (midpoints between class values)
            bounds = [0.5, 1.5, 2.5, 3.5]
            norm = mcolors.BoundaryNorm(bounds, cmap.N)

            # Handle NaN values in the 3-class map for plotting
            # Create a masked array where NaN values are masked
            three_class_map_masked = np.ma.masked_where(np.isnan(three_class_map), three_class_map)

            # 5. Create a figure and axes
            fig, ax = plt.subplots(figsize=(10, 8))

            # 6. Display the calculated hillshade
            rasterio.plot.show(
                hillshade_array,
                ax=ax,
                cmap='gray',
                transform=clipped_dsm_transform # Use the transform from the clipped DSM
            )
            print("Hillshade displayed.")

            # 7. Overlay the 3-class map
            im = rasterio.plot.show(
                three_class_map_masked, # Use the masked array
                ax=ax,
                cmap=cmap,
                norm=norm,
                transform=clipped_dsm_transform, # Use the transform from the clipped DSM
                alpha=0.6, # Set alpha for semi-transparency
                interpolation='nearest' # Use nearest neighbor for discrete classes
            )
            print("3-class map overlaid.")

            # 8. Add a title and labels
            ax.set_title('Productivity Classes on DSM Hillshade')
            ax.set_xlabel('Easting') # Assuming projected CRS, adjust if necessary
            ax.set_ylabel('Northing') # Assuming projected CRS, adjust if necessary

            # 9. Add a legend or colorbar
            # Create a colorbar with discrete ticks and labels
            cbar = fig.colorbar(im.get_images()[0], ax=ax, ticks=class_values)
            cbar.ax.set_yticklabels(class_labels)
            cbar.set_label('Productivity Class')

            # 10. Display the plot
            plt.show()
            print("Visualization displayed.")
        else:
            print("Skipping visualization due to hillshade calculation error.")

Required data (3-class map, clipped DSM, or target transform) is not fully available. Skipping visualization.


## Summary:

### Data Analysis Key Findings

*   The process involved attempting to upload, load, reproject, align, and clip DSM and TPI raster data to analyze their relationship with productivity indices.
*   File upload widgets for DSM and TPI were successfully created, but no files were uploaded for either.
*   Consequently, the loading of DSM and TPI data resulted in zero datasets being loaded.
*   Due to the absence of loaded original datasets, the reprojection and alignment step for DSM and TPI was skipped.
*   The clipping of DSM and TPI to the AOI was also skipped because the aligned datasets were not available.
*   The analysis of the relationship between productivity indices, DSM, and TPI, including correlation, scatter plots, and zonal statistics, could not be performed due to the lack of clipped DSM and TPI data and potentially other required variables like the composite productivity index and 3-class map.
*   The visualization of the combined data (overlaying productivity classes on a DSM hillshade) was also skipped because the necessary clipped DSM and 3-class map were not available.

### Insights or Next Steps

*   Ensure that DSM and TPI raster files are successfully uploaded before proceeding with subsequent analysis steps.
*   Verify that the composite productivity index and the 3-class productivity map are successfully generated and available from previous steps, as they are crucial inputs for the relationship analysis and visualization.
